<a href="https://colab.research.google.com/github/VectorJamo/Deep-Learning/blob/main/Binary_Classification_Using_Support_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# SVM classifies data by defining a hyperplane with maximal margin that separates the the data clusters
import numpy as py
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
dataset = pd.read_csv('/content/Student Depression Dataset.csv')

In [18]:
dataset.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [19]:
# We will extract the following columns to train our model: Gender, Academic Pressure, CGPA, Study Satisfaction, Suicidial Thoughts, Financial Stress,
# and Family history of Mental illness
dataset = dataset[['Gender', 'Academic Pressure', 'CGPA', 'Study Satisfaction', 'Have you ever had suicidal thoughts ?', 'Financial Stress', 'Family History of Mental Illness']]
dataset

,Gender,Academic Pressure,CGPA,Study Satisfaction,Have you ever had suicidal thoughts ?,Financial Stress,Family History of Mental Illness
0,Male,5.0,8.97,2.0,Yes,1.0,No
1,Female,2.0,5.90,5.0,No,2.0,Yes
2,Male,3.0,7.03,5.0,No,1.0,Yes
3,Female,3.0,5.59,2.0,Yes,5.0,Yes
4,Female,4.0,8.13,3.0,Yes,1.0,No
...,...,...,...,...,...,...,...
27896,Female,5.0,5.75,5.0,Yes,1.0,Yes
27897,Male,2.0,9.40,3.0,No,3.0,Yes
27898,Male,3.0,6.61,4.0,No,2.0,No
27899,Female,5.0,6.88,2.0,Yes,5.0,No
